In [1]:
from datetime import datetime

In [2]:
class WidgetExc(Exception):
    msg = 'generic widget exce' #class attr , def internal msg
    
    def __init__(self, *args, custom_msg = None):
        super().__init__(args)
        if args:
            self.msg = args[0]
        self.custom_msg = custom_msg if custom_msg is not None else self.msg
        
    def log_exc(self):
        exc = {
            "type": type(self).__name__, #type is mentioned here so that it can take the class name
            "msg": self.msg,
            "args": self.args[1:]
        }
        
        print(f'EXCEPTION: {datetime.utcnow().isoformat()}: {exc}')
        

In [3]:
ex1 = WidgetExc('some custome msg', 10, 100)
ex2 = WidgetExc(custom_msg='A custom user msg')

In [6]:
ex1.log_exc()

EXCEPTION: 2021-02-24T20:51:44.973503: {'type': 'WidgetExc', 'msg': 'some custome msg', 'args': ()}


In [7]:
ex2.log_exc()

EXCEPTION: 2021-02-24T20:51:56.943915: {'type': 'WidgetExc', 'msg': 'generic widget exce', 'args': ()}


In [8]:
class SupplierExc(WidgetExc):
    msg = 'supplier exception'
    
class NotManuExc(SupplierExc):
    msg = 'Widget is no longer manu by supplier'
    
class ProdDelayExc(SupplierExc):
    msg = 'widget production has been delayed by the manu'
    
class ShippingDelExc(SupplierExc):
    msg = 'Widget shipping has been dealyed by the supplier'

class CheckOutExc(WidgetExc):
    msg = 'Check out exception'

class InventExc(CheckOutExc):
    msg = 'check out inventory exception'
    
class OutOfStockExc(InventExc):
    msg = 'inventory out of stock'
    
class PricingExc(CheckOutExc):
    msg = 'checkout pricing exception'
    
class InvalidCCExc(PricingExc):
    msg = 'invalid checkout coupon code'
    
class CannotStackExc(PricingExc):
    msg = 'cannot stack coupon code'
    


In [9]:
try:
    raise CannotStackExc()
except CannotStackExc as ex:
    ex.log_exc()
    raise 

EXCEPTION: 2021-02-24T20:57:59.996272: {'type': 'CannotStackExc', 'msg': 'cannot stack coupon code', 'args': ()}


CannotStackExc: ()

In [11]:
import json

In [10]:
from http import HTTPStatus

In [12]:
class WidgetExc(Exception):
    msg = 'generic widget exce' #class attr , def internal msg
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    def __init__(self, *args, custom_msg = None):
        super().__init__(args)
        if args:
            self.msg = args[0]
        self.custom_msg = custom_msg if custom_msg is not None else self.msg
        
    def log_exc(self):
        exc = {
            "type": type(self).__name__, #type is mentioned here so that it can take the class name
            "msg": self.msg,
            "args": self.args[1:]
        }
        
        print(f'EXCEPTION: {datetime.utcnow().isoformat()}: {exc}')
        
    
    def to_json(self):
        response = {
            'code': self.http_status.value,
            'msg': '{}: {}'.format(self.http_status.phrase, self.custom_msg),
            'category': type(self).__name__,
            'time_utc':datetime.utcnow().isoformat()
        }
        return json.dumps(response)
    

In [13]:
e = WidgetExc('same custom msg for log and user')

In [15]:
e.log_exc()

EXCEPTION: 2021-02-24T21:03:26.644250: {'type': 'WidgetExc', 'msg': 'same custom msg for log and user', 'args': ()}


In [17]:
json.loads(e.to_json())

{'code': 500,
 'msg': 'Internal Server Error: same custom msg for log and user',
 'category': 'WidgetExc',
 'time_utc': '2021-02-24T21:03:50.936902'}

In [18]:
e = WidgetExc('custom internal msg', custom_msg= 'custom user msg')

In [19]:
e.log_exc()

EXCEPTION: 2021-02-24T21:04:47.304873: {'type': 'WidgetExc', 'msg': 'custom internal msg', 'args': ()}


In [20]:
e.to_json()

'{"code": 500, "msg": "Internal Server Error: custom user msg", "category": "WidgetExc", "time_utc": "2021-02-24T21:04:56.005408"}'

In [22]:
try:
    raise WidgetExc('custom error msg')
except WidgetExc as ex:
    print(repr(ex.__traceback__))

In [23]:
import traceback

In [25]:
try:
    raise ValueError
except ValueError:
    try:
        raise WidgetExc('custome msg')
    except WidgetExc as ex:
        tb = traceback.TracebackException.from_exception(ex)
        print(list(tb.format()))

['Traceback (most recent call last):\n', '  File "<ipython-input-25-4755370c58d1>", line 2, in <module>\n    raise ValueError\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "<ipython-input-25-4755370c58d1>", line 5, in <module>\n    raise WidgetExc(\'custome msg\')\n', "WidgetExc: ('custome msg',)\n"]


In [26]:
try:
    raise ValueError
except ValueError:
    try:
        raise WidgetExc('custome msg')
    except WidgetExc as ex:
        tb = traceback.TracebackException.from_exception(ex)
        tb = ''.join(tb.format())
        print(tb)

Traceback (most recent call last):
  File "<ipython-input-26-d14c63b87aa1>", line 2, in <module>
    raise ValueError
ValueError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-26-d14c63b87aa1>", line 5, in <module>
    raise WidgetExc('custome msg')
WidgetExc: ('custome msg',)



In [28]:
try:
    raise ValueError
except ValueError:
    raise WidgetExc('custome msg')
   

WidgetExc: ('custome msg',)

In [31]:
class WidgetExc(Exception):
    msg = 'generic widget exce' #class attr , def internal msg
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    def __init__(self, *args, custom_msg = None):
        super().__init__(args)
        if args:
            self.msg = args[0]
        self.custom_msg = custom_msg if custom_msg is not None else self.msg
        
    @property
    def traceback(self):
        return traceback.TracebackException.from_exception(self).format()
    
        
    def log_exc(self):
        exc = {
            "type": type(self).__name__, #type is mentioned here so that it can take the class name
            "msg": self.msg,
            "args": self.args[1:],
            "traceback": list(self.traceback)
        }
        
        print(f'EXCEPTION: {datetime.utcnow().isoformat()}: {exc}')
        
    
    def to_json(self):
        response = {
            'code': self.http_status.value,
            'msg': '{}: {}'.format(self.http_status.phrase, self.custom_msg),
            'category': type(self).__name__,
            'time_utc':datetime.utcnow().isoformat()
        }
        return json.dumps(response)
    

In [33]:
try:
    raise WidgetExc()
except WidgetExc as ex:
    ex.log_exc()
    print('--------------')
    print(ex.to_json())

EXCEPTION: 2021-02-24T21:17:03.481563: {'type': 'WidgetExc', 'msg': 'generic widget exce', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "<ipython-input-33-b936ba42604c>", line 2, in <module>\n    raise WidgetExc()\n', 'WidgetExc: ()\n']}
--------------
{"code": 500, "msg": "Internal Server Error: generic widget exce", "category": "WidgetExc", "time_utc": "2021-02-24T21:17:03.482560"}


In [34]:
try:
    a = 1/0
except ZeroDivisionError:
    try:
        raise WidgetExc()
    except WidgetExc as ex:
        print(''.join(ex.traceback))

Traceback (most recent call last):
  File "<ipython-input-34-6c709a45c8a7>", line 2, in <module>
    a = 1/0
ZeroDivisionError: division by zero

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-34-6c709a45c8a7>", line 5, in <module>
    raise WidgetExc()
WidgetExc: ()



In [35]:
class SupplierExc(WidgetExc):
    msg = 'supplier exception'
    
class NotManuExc(SupplierExc):
    msg = 'Widget is no longer manu by supplier'
    
class ProdDelayExc(SupplierExc):
    msg = 'widget production has been delayed by the manu'
    
class ShippingDelExc(SupplierExc):
    msg = 'Widget shipping has been dealyed by the supplier'

class CheckOutExc(WidgetExc):
    msg = 'Check out exception'

class InventExc(CheckOutExc):
    msg = 'check out inventory exception'
    
class OutOfStockExc(InventExc):
    msg = 'inventory out of stock'
    
class PricingExc(CheckOutExc):
    msg = 'checkout pricing exception'
    
class InvalidCCExc(PricingExc):
    msg = 'invalid checkout coupon code'
    
class CannotStackExc(PricingExc):
    msg = 'cannot stack coupon code'

In [36]:
e = InvalidCCExc(
    'user tried to pull fast one upon us', 
    custom_msg='sorry this coupon is not valid')

In [37]:
e.log_exc()

EXCEPTION: 2021-02-24T21:19:24.239719: {'type': 'InvalidCCExc', 'msg': 'user tried to pull fast one upon us', 'args': (), 'traceback': ["InvalidCCExc: ('user tried to pull fast one upon us',)\n"]}


In [38]:
e.to_json()

'{"code": 500, "msg": "Internal Server Error: sorry this coupon is not valid", "category": "InvalidCCExc", "time_utc": "2021-02-24T21:19:50.671624"}'

In [41]:
try:
    raise ValueError
except ValueError:
    try:
        raise InvalidCCExc(
            'user tried to pull fast one upon us', 
            custom_msg='sorry this coupon is not valid')
        
    except InvalidCCExc as ex:
        ex.log_exc()
        print('------------')
        print(ex.to_json)
        print('---------')
        print(''.join(ex.traceback))

EXCEPTION: 2021-02-24T21:22:43.665783: {'type': 'InvalidCCExc', 'msg': 'user tried to pull fast one upon us', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "<ipython-input-41-fd142d380331>", line 2, in <module>\n    raise ValueError\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "<ipython-input-41-fd142d380331>", line 5, in <module>\n    raise InvalidCCExc(\n', "InvalidCCExc: ('user tried to pull fast one upon us',)\n"]}
------------
<bound method WidgetExc.to_json of InvalidCCExc(('user tried to pull fast one upon us',))>
---------
Traceback (most recent call last):
  File "<ipython-input-41-fd142d380331>", line 2, in <module>
    raise ValueError
ValueError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-41-fd142d380331>", line 5, in <module>
    raise InvalidCCExc(
InvalidCCExc: (